### The file d_swap.csv contains the daily swap rates r_{kt} for eight maturities T_k = 1, 2, 3, 4, 5, 7, 10 and 30 years from July 3, 2000 to October 28, 2016.

\begin{equation}
d_k - \hat{\mu}_k = \sum_{j=1}^{8} \hat{a}_{kj} Y_j
\label{eq:approx}
\end{equation}

### The ratio 
### $$\frac{\hat{a}_{kj}^2}{\sum_{i=1}^{8} \hat{a}_{ki}^2}$$ 
### represents the proportion that the \( j_{th} \) principal component contributes to the variance of the daily changes in the swap rate with maturity \( T_{k} \). Compute this ratio for the first three principal components and each swap rate.


In [1]:
# Import relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# load the data for dswap
df = pd.read_csv('d_swap.csv')

n = len(df)
p = len(df.columns[1:])
S = np.zeros((p,n))

i = 0
for col in df.columns[1:]:
    S[i,:]=df[col].astype(float)
    i +=1

# First difference of the matrix
X = np.diff(S,axis=1)
mu = np.outer(np.mean(X,axis=1),np.ones((n-1,1)))
X = X - mu


# Compute single value decomposition of matrix X
u,s,pt = np.linalg.svd(X,full_matrices=False)


# Compute variance percentage using single value diagonal matrix
proportion_matrix = (s**2/np.sum(s**2)).reshape(1,8)
proportion_matrix_df = pd.DataFrame(proportion_matrix, columns = ['PC ' + str(i+1) for i in range(u.shape[0])], index = ['Variance'])*100



# Represent data in terms of principal components and vice-a-versa
Y = u.T@X       # Principal components in terms of data
X_pca= u@Y      # Data in terms of principal components


# Percentage contribution of each principal component to each maturity
contribution_df = pd.DataFrame(u, columns = ['a'+ str(i+1) for i in range(u.shape[1])], index = ['1y', '2y', '3y', '4y', '5y', '7y', '10y', '30y'])
contribution_df = contribution_df**2
contribution_df = contribution_df.apply(lambda row: row/row.sum(), axis = 1)*100
contribution_df.index.name = 'Maturity'

In [2]:
print('Variance Contribution of each PC')
round(proportion_matrix_df,3)

Variance Contribution of each PC


,PC 1,PC 2,PC 3,PC 4,PC 5,PC 6,PC 7,PC 8
Variance,89.679,7.769,1.425,0.51,0.251,0.169,0.102,0.095


In [3]:
print('\n')
print('Percentage Contribution of first 3 PCs to the swap rates')
print('\n')
round(contribution_df.iloc[:,0:3],3)



Percentage Contribution of first 3 PCs to the swap rates




,a1,a2,a3
Maturity,,,
1y,3.944,21.681,38.024
2y,9.815,18.965,3.767
3y,13.103,8.683,1.442
4y,15.107,1.961,6.582
5y,16.478,0.015,13.062
7y,16.214,3.188,4.860
10y,15.022,13.081,0.118
30y,10.316,32.426,32.144


## Monthly Swap Rates
### The file m_swap.csv contains the monthly swap rates r_{kt} for eight maturities T_k = 1, 2, 3, 4, 5, 7, 10 and 30 years from July 2000 to October 2016.

In [4]:
# Import relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Loading monthly swap data
df = pd.read_csv('m_swap.csv')

# building a matrix
n = len(df)
p = len(df.columns[1:])
S = np.zeros((p,n))
i = 0
for col in df.columns[1:]:
    S[i,:]=df[col].astype(float)
    i +=1

# Centering the data
X = np.diff(S,axis=1)
mu = np.outer(np.mean(X,axis=1),np.ones((n-1,1)))
X_diff = X - mu

## PCA Using Sample Covariance Matrix

In [6]:
# Computing covariance
cov_X = (X_diff@X_diff.T)/(X_diff.shape[1]-1)

# Computing eigenvalues and eigenvectors of cov_X
eigvals_cov, eigvecs_cov = np.linalg.eig(cov_X)

# Sort the eigenvalues in descending order
sorted_indices = np.argsort(eigvals_cov)[::-1]

# Sort eigenvalues and eigenvectors
sorted_eigvals = eigvals_cov[sorted_indices]
sorted_eigvecs = eigvecs_cov[:, sorted_indices]


PCA_cov_df = pd.DataFrame(sorted_eigvecs, columns = ['PC' + str(i+1) for i in range(sorted_eigvecs.shape[1])], index = ['1y', '2y', '3y', '4y', '5y', '7y', '10y', '30y'] )
print('Sorted Eigenvectors for PCA using Covariance Matrix')
print('\n')
round(PCA_cov_df,4)

Sorted Eigenvectors for PCA using Covariance Matrix




,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8
1y,0.2463,0.6225,0.5662,0.4269,0.2059,0.0766,0.0226,0.0147
2y,0.3242,0.4178,0.0099,-0.4549,-0.5652,-0.3749,-0.1937,-0.1252
3y,0.3672,0.2129,-0.2484,-0.3542,0.0714,0.4907,0.4764,0.3999
4y,0.3883,0.0525,-0.3220,-0.0740,0.4142,0.2085,-0.2550,-0.6762
5y,0.3955,-0.0668,-0.2877,0.1753,0.3073,-0.3938,-0.4044,0.5591
7y,0.3920,-0.2216,-0.1215,0.3729,-0.1419,-0.4039,0.6391,-0.2275
10y,0.3763,-0.3391,0.0968,0.3317,-0.5309,0.4920,-0.3120,0.0516
30y,0.3112,-0.4704,0.6380,-0.4472,0.2549,-0.0949,0.0264,0.0043


In [7]:
# Variance explained by top 3 Principal Components using Covariance Matrix
variance_explained_cov = (eigvals_cov[0]+eigvals_cov[1]+eigvals_cov[2])/sum(eigvals_cov)
print("Variance explained by top 3 Principal Components using Covariance Matrix: ", round(variance_explained_cov*100,3),'%')

Variance explained by top 3 Principal Components using Covariance Matrix:  99.489 %


## PCA Using Sample Correlation Matrix

In [8]:
# Calculating standard deviations of each variable
std_devs = np.sqrt(np.diag(cov_X))

# Divide each element of the corrariance matrix by the product of standard deviations to get Correlation matrix
corr_X = cov_X / np.outer(std_devs, std_devs)

# Computing eigen values and eigen vectors
eigvals_corr, eigvecs_corr = np.linalg.eig(corr_X)

# Sort the eigenvalues in descending order
sorted_indices = np.argsort(eigvals_corr)[::-1]

# Sort eigenvalues and eigenvectors
sorted_eigvals = eigvals_corr[sorted_indices]
sorted_eigvecs = eigvecs_corr[:, sorted_indices]

PCA_corr_df = pd.DataFrame(sorted_eigvecs, columns = ['PC' + str(i+1) for i in range(sorted_eigvecs.shape[1])], index = ['1y', '2y', '3y', '4y', '5y', '7y', '10y', '30y'] )
print('Sorted Eigenvectors for PCA using Correlation Matrix')
print('\n')
round(PCA_corr_df,4)

Sorted Eigenvectors for PCA using Correlation Matrix




,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8
1y,0.2885,0.6482,0.5524,0.3920,0.1821,0.0648,0.0196,0.0071
2y,0.3509,0.3884,-0.0538,-0.4959,-0.5587,-0.3533,-0.1895,-0.0662
3y,0.3722,0.1686,-0.2838,-0.3302,0.1280,0.5245,0.5325,0.2615
4y,0.3771,0.0159,-0.3270,-0.0355,0.4271,0.1597,-0.4324,-0.5954
5y,0.3762,-0.0898,-0.2799,0.2004,0.2890,-0.4138,-0.2395,0.6479
7y,0.3698,-0.2237,-0.1181,0.3795,-0.1596,-0.3851,0.5854,-0.3725
10y,0.3594,-0.3297,0.0854,0.3416,-0.5374,0.4927,-0.3040,0.1189
30y,0.3249,-0.4832,0.6363,-0.4350,0.2408,-0.0897,0.0264,-0.0011


In [9]:
# Variance explained by top 3 Principal Components using Correlation Matrix
variance_explained_corr = (eigvals_corr[0]+eigvals_corr[1]+eigvals_corr[2])/sum(eigvals_corr)

print("Variance explained by top 3 Principal Components using Correlation Matrix: ", round(variance_explained_corr*100,3),'%')

Variance explained by top 3 Principal Components using Correlation Matrix:  99.467 %


# Comparison of Monthly and Daily Swaps

In [10]:
# PCA factor loadings from covariance 
inv_eigvec_cov = np.linalg.inv(eigvecs_cov).T

# Tabulate factor loadings into a dataframe
contribution_cov = pd.DataFrame(inv_eigvec_cov, columns = ['a'+ str(i+1) for i in range(inv_eigvec_cov.shape[1])], index = ['1y', '2y', '3y', '4y', '5y', '7y', '10y', '30y'])
contribution_cov = contribution_cov**2
contribution_cov = contribution_cov.apply(lambda row: row/row.sum(), axis = 1)*100
contribution_cov.index.name = 'Maturity'
print('Percentage Contribution of first 3 PCs from covariance matrix to the swap rates')
round(contribution_cov.iloc[:,:3],3)

Percentage Contribution of first 3 PCs from covariance matrix to the swap rates


,a1,a2,a3
Maturity,,,
1y,6.066,38.753,32.056
2y,10.513,17.453,0.010
3y,13.484,4.531,6.170
4y,15.080,0.275,10.367
5y,15.645,0.447,8.274
7y,15.367,4.909,1.476
10y,14.161,11.502,0.937
30y,9.684,22.131,40.711


In [11]:
# PCA factor loadings from correlation matrix
inv_eigvec_corr = np.linalg.inv(eigvecs_corr).T

# Tabulate factor loadings into a dataframe
contribution_corr = pd.DataFrame(inv_eigvec_corr, columns = ['a'+ str(i+1) for i in range(inv_eigvec_corr.shape[1])], index = ['1y', '2y', '3y', '4y', '5y', '7y', '10y', '30y'])
contribution_corr = contribution_corr**2
contribution_corr = contribution_corr.apply(lambda row: row/row.sum(), axis = 1)*100
contribution_corr.index.name = 'Maturity'
print('Percentage Contribution of first 3 PCs from correlation matrix to the swap rates')
round(contribution_corr.iloc[:,:3],3)

Percentage Contribution of first 3 PCs from correlation matrix to the swap rates


,a1,a2,a3
Maturity,,,
1y,8.323,42.019,30.511
2y,12.310,15.082,0.290
3y,13.851,2.842,8.056
4y,14.219,0.025,10.691
5y,14.153,0.806,7.836
7y,13.672,5.006,1.394
10y,12.918,10.869,0.729
30y,10.554,23.351,40.493


Analysis of Results

PC1 can be interpreted as the Parallel Shift Component. Its factor loadings are roughly constant over all maturities.

PC2 can be interpreted as the Tilt Component. Changes in short maturity and long maturity swap rates have opposite signs in this component.

PC3 can be interpreted as the Curvature Component. The factor loadings are negative for the midterm rates and highly positive for the short term and long term rates depicting a convex curvature for the graph of rates vs maturities.

On the basis of sampling frequency, we can observe that the PC1 for the daily frequency swap rates contributes significantly less to the variance of the short term swap rates compared to the PC1 for the monthly frequency swap rates. The contribution to the variances of the mid-term and long term maturities by PC1 is approximately the same for both daily and monthly frequency swap rates.

Similarly, the PC2 contributes significantly less for the daily frequency swap rates for 1y rates but it contributes higher for 2y and 3y rates. The variance contribution for the mid-term rates in similar across both sampling frequencies, while the contribution is very high for long-term maturity for daily frequency swap rates.

For PC3, the variance contribution is higher for the daily frequency rates across all maturities barring only 4y and 10y maturities. The variance is the same for both frequency swap rates for 10y rates while the variance contribution is higher for monthly frequency rates by PC3 at 4y maturity.  
